# 生成网络

循环神经网络 (RNN) 及其门控单元变体，例如长短期记忆单元 (LSTM) 和门控循环单元 (GRU) 提供了一种语言建模机制，即它们可以学习词序并为序列中的下一个词提供预测.这允许我们将 RNN 用于**生成任务**，例如普通文本生成、机器翻译，甚至图像字幕。

在我们在上一个单元中讨论的 RNN 架构中，每个 RNN 单元产生下一个隐藏状态作为输出。但是，我们也可以为每个循环单元添加另一个输出，这将允许我们输出一个**序列**（其长度等于原始序列）。此外，我们可以使用在每一步都不接受输入的 RNN 单元，只取一些初始状态向量，然后产生一系列输出。

这允许如下图所示的不同神经架构：

![显示常见循环神经网络模式的图像。](./images/unreasonable-effectiveness-of-rnn.jpeg)
*图片来自 [Andrej Karpaty](http://karpathy.github.io) 的博客文章 [循环神经网络的不合理有效性](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) /)*

* **一对一**是传统的一输入一输出的神经网络
* **一对多**是一种生成架构，它接受一个输入值，并生成一系列输出值。例如，如果我们想训练 **image captioning** 网络来生成图片的文本描述，我们可以将图片作为输入，将其通过 CNN 获取隐藏状态，然后让循环链生成标题词——逐字逐句
* **多对一**对应我们在上一单元中描述的RNN架构，例如文本分类
* **Many-to-many**，或**sequence-to-sequence**对应于诸如**机器翻译**之类的任务，我们首先让RNN将输入序列中的所有信息收集到隐藏状态，另一个 RNN 链将此状态展开到输出序列中。

在本单元中，我们将专注于帮助我们生成文本的简单生成模型。为简单起见，让我们构建**字符级网络**，逐个字母生成文本。在训练过程中，我们需要取一些文本语料，并将其拆分为字母序列。

In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## 构建字符词汇

为了构建字符级生成网络，我们需要将文本拆分为单个字符而不是单词。 这可以通过定义不同的标记器来完成：

In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.Vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.stoi['a']}")
print(f"Character with code 13 is {vocab.itos[13]}")

Vocabulary size = 84
Encoding of 'a' is 4
Character with code 13 is h


让我们看看如何对数据集中的文本进行编码的示例：

In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([43,  4, 11, 11,  2, 26,  5, 23,  2, 38,  3,  4, 10,  9,  2, 31, 11,  4,
        21,  2, 38,  4, 14, 25,  2, 34,  8,  5,  6,  2,  5, 13,  3,  2, 38, 11,
         4, 14, 25,  2, 55, 37,  3, 15,  5,  3, 10,  9, 56,  2, 37,  3, 15,  5,
         3, 10,  9,  2, 29,  2, 26, 13,  6, 10,  5, 29,  9,  3, 11, 11,  3, 10,
         9, 27,  2, 43,  4, 11, 11,  2, 26,  5, 10,  3,  3,  5, 58,  9,  2, 12,
        21,  7,  8, 12, 11,  7,  8, 18, 61, 22,  4,  8, 12,  2,  6, 19,  2, 15,
        11,  5, 10,  4, 29, 14, 20,  8,  7, 14,  9, 27,  2,  4, 10,  3,  2,  9,
         3,  3,  7,  8, 18,  2, 18, 10,  3,  3,  8,  2,  4, 18,  4,  7,  8, 23])

## 训练一个生成式 RNN

我们将训练 RNN 生成文本的方式如下。在每一步中，我们将获取一个长度为 nchars 的字符序列，并要求网络为每个输入字符生成下一个输出字符：

![图像显示了单词“HELLO”的示例 RNN 生成。](./images/rnn-generate.png)

根据实际场景，我们可能还希望包含一些特殊字符，例如 *end-of-sequence* `<eos>`。在我们的例子中，我们只想训练网络生成无限的文本，因此我们将每个序列的大小固定为等于 `nchars` 个标记。因此，每个训练示例将包含 `nchars` 输入和 `nchars` 输出（它们是向左移动一个符号的输入序列）。 Minibatch 将由几个这样的序列组成。

我们生成小批量的方法是获取每个长度为 `l` 的新闻文本，并从中生成所有可能的输入-输出组合（会有 `l-nchars` 这样的组合）。它们将形成一个 minibatch，并且在每个训练步骤中 minibatch 的大小都不同。

In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[43,  4, 11,  ..., 18, 61, 22],
         [ 4, 11, 11,  ..., 61, 22,  4],
         [11, 11,  2,  ..., 22,  4,  8],
         ...,
         [37,  3, 15,  ...,  4, 18,  4],
         [ 3, 15,  5,  ..., 18,  4,  7],
         [15,  5,  3,  ...,  4,  7,  8]]),
 tensor([[ 4, 11, 11,  ..., 61, 22,  4],
         [11, 11,  2,  ..., 22,  4,  8],
         [11,  2, 26,  ...,  4,  8, 12],
         ...,
         [ 3, 15,  5,  ..., 18,  4,  7],
         [15,  5,  3,  ...,  4,  7,  8],
         [ 5,  3, 10,  ...,  7,  8, 23]]))

现在让我们定义生成器网络。 它可以基于我们在上一单元中讨论的任何循环单元（简单、LSTM 或 GRU）。 在我们的示例中，我们将使用 LSTM。

因为网络以字符为输入，词汇量很小，我们不需要嵌入层，one-hot-encoded输入可以直接进入LSTM单元。 但是，因为我们将字符数作为输入传递，所以在传递给 LSTM 之前，我们需要对它们进行单热编码。 这是通过在 `forward` 传递期间调用 `one_hot` 函数来完成的。 输出编码器将是一个线性层，它将隐藏状态转换为单热编码输出。

In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

在训练期间，我们希望能够对生成的文本进行采样。 为此，我们将定义`generate` 函数，该函数将从初始字符串`start` 开始生成长度为`size` 的输出字符串。

它的工作方式如下。 首先，我们将整个起始字符串通过网络，并获取输出状态 `s` 和下一个预测字符 `out`。 由于 `out` 是 one-hot 编码，我们使用 `argmax` 来获取字符 `nc` 在词汇表中的索引，并使用 `itos` 找出实际字符并将其附加到结果字符列表中` 字符`。 这个生成一个字符的过程被重复`size`次以生成所需数量的字符。

In [6]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.itos[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

现在让我们开始训练吧！ 训练循环与我们之前的所有示例几乎相同，但我们每 1000 次打印采样生成的文本而不是准确性。

需要特别注意我们计算损失的方式。 我们需要计算给定单热编码输出 `out` 和预期文本 `text_out`（字符索引列表）的损失。 幸运的是，`cross_entropy` 函数期望非标准化网络输出作为第一个参数，类编号作为第二个参数，这正是我们所拥有的。 它还对小批量大小执行自动平均。

我们还通过 `samples_to_train` 样本限制训练，以免等待太久。 我们鼓励您进行试验并尝试更长的训练时间，可能是几个时期（在这种情况下，您需要围绕此代码创建另一个循环）。

In [7]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.414118766784668
today ttettettettettettettettettettettettettettettettettettettettettettettettettettettettettettettettettet
Current loss = 2.051412582397461
today and a the and a the and a the and a the and a the and a the and a the and a the and a the and a the 
Current loss = 1.6169160604476929
today and the US the US the US the US the US the US the US the US the US the US the US the US the US the U
Current loss = 2.3653104305267334
today to the United State and the United State and the United State and the United State and the United St
Current loss = 1.6601594686508179
today to the stock a start to the stock a start to the stock a start to the stock a start to the stock a s
Current loss = 1.6862702369689941
today and the International and the International and the International and the International and the Inte
Current loss = 1.9996715784072876
today and a for the the the the the the the the the the the the the the the the the the the the the the th
Current loss = 

这个例子已经生成了一些非常好的文本，但可以通过以下几种方式进一步改进：
* **更好的小批量生成**。我们准备训练数据的方式是从一个样本生成一个小批量。这并不理想，因为 minibatch 的大小都不同，有些甚至无法生成，因为文本比 nchars 小。此外，小批量不能充分加载 GPU。更明智的做法是从所有样本中获取一大块文本，然后生成所有输入-输出对，将它们打乱，并生成大小相等的小批量。
* **多层 LSTM**。尝试 2 或 3 层 LSTM 单元是有意义的。正如我们在上一单元中提到的，LSTM 的每一层都从文本中提取某些模式，在字符级生成器的情况下，我们可以期望较低的 LSTM 级别负责提取音节，而更高的级别 - 用于单词和单词组合。这可以通过将 number-of-layers 参数传递给 LSTM 构造函数来简单地实现。
* 您可能还想试验 **GRU 单元**，看看哪些性能更​​好，以及 **不同的隐藏层大小**。太大的隐藏层可能会导致过拟合（例如网络将学习精确的文本），而较小的尺寸可能不会产生好的结果。

## 软文本生成和温度

在之前的 `generate` 定义中，我们总是将概率最高的字符作为生成文本中的下一个字符。这导致文本经常在相同的字符序列之间“循环”一次又一次，如下例所示：
``
今天的第二个公司和第二个公司......
``

但是，如果我们查看下一个字符的概率分布，可能几个最高概率之间的差异并不大，例如一个字符的概率为 0.2，另一个为 0.19，等等。例如，在查找序列 '*play*' 中的下一个字符时，下一个字符同样可以是空格或 **e**（如词*玩家*）。

这使我们得出结论，选择概率较高的字符并不总是“公平”的，因为选择第二高的字符可能仍然会让我们得到有意义的文本。从网络输出给出的概率分布中**采样**字符更为明智。

这种采样可以使用实现所谓的**多项分布**的“多项式”函数来完成。实现此 **soft** 文本生成的函数定义如下：

In [8]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.itos[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today for the company on Wednesday the manage with the United States of a worker the stati and a the United States who one the bell with the first with have the rester of the company and A government to a start and the United States and worker the sear the first worken computer of a starling the for his t

--- Temperature = 0.8
Today An Brityson, and flagiest Victoria it country end It cabres slower. The Churm Operal a food of and P9,000 Polaturaplic (GS Operation Athlicablic on have a says waber the bung more the on Wednerre world Wednesday, their bagged in the say the the bushn this growth would and and enex chorn at softwarn 

--- Temperature = 1.0
Today purchase exactiegs purchers in Clan #39;s a collers and Russian Devell 2 mixis Wobis nem fall and stafiightsoal, bidlied a faru sharcun sayers it oil from a coubrebates most game with the will call, the security sells that home the has silitor of Ireani Tussian swarneted fell run for comesaging foug

--- Temper

我们又引入了一个称为**温度**的参数，用于指示我们应该坚持最高概率的程度。 如果温度为 1.0，我们进行公平多项式采样，当温度达到无穷大时 - 所有概率变得相等，我们随机选择下一个字符。 在下面的示例中，我们可以观察到，当我们将温度升高太多时，文本变得毫无意义，而当它变得接近 0 时，它类似于“循环”硬生成的文本。